## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-20-13-09-07 +0000,nyt,Trump Doubles Down on Greenland Threats in Flu...,https://www.nytimes.com/2026/01/20/world/europ...
1,2026-01-20-13-07-00 +0000,wsj,U.S. Natural Gas Soars As Extreme Weather Show...,https://www.wsj.com/finance/commodities-future...
2,2026-01-20-13-04-00 +0000,wsj,Israel Destroys Parts of U.N. Agency Headquart...,https://www.wsj.com/world/middle-east/israel-d...
3,2026-01-20-13-03-08 +0000,nyt,‘Make America Go Away’ Caps Surge in Popularit...,https://www.nytimes.com/2026/01/20/us/make-ame...
4,2026-01-20-12-57-19 +0000,nypost,Warner Bros. Discovery accepts Netflix’s amend...,https://nypost.com/2026/01/20/business/warner-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,77
2,greenland,28
45,new,16
90,ice,14
89,minnesota,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
130,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...,158
69,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,148
13,2026-01-20-12-34-10 +0000,nypost,House Speaker Mike Johnson tells UK Parliament...,https://nypost.com/2026/01/20/us-news/house-sp...,143
126,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/elections/why-an-...,143
72,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
130,158,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...
11,69,2026-01-20-12-44-01 +0000,nypost,New Hampshire bishop warns of ‘new era of mart...,https://nypost.com/2026/01/20/us-news/new-hamp...
245,56,2026-01-19-19-24-57 +0000,nyt,High-Speed Train Crash in Spain Leaves at Leas...,https://www.nytimes.com/live/2026/01/18/world/...
72,52,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
205,38,2026-01-19-22-58-10 +0000,nypost,"LI school board boss, who launched fight to sa...",https://nypost.com/2026/01/19/us-news/li-schoo...
176,36,2026-01-20-00-50-49 +0000,nypost,Protesters gather outside Atlanta’s Emory Univ...,https://nypost.com/2026/01/19/us-news/silent-p...
71,34,2026-01-20-10-30-00 +0000,wsj,Heard on the Street: Conflicting interests mak...,https://www.wsj.com/economy/housing/to-make-ho...
116,32,2026-01-20-08-10-29 +0000,nypost,Trump calls UK’s Chagos deal with Mauritius an...,https://nypost.com/2026/01/20/us-news/trump-ca...
216,29,2026-01-19-22-17-41 +0000,nypost,NY Dems in House primary refuse to answer Mamd...,https://nypost.com/2026/01/19/us-news/ny-dems-...
143,29,2026-01-20-03-15-28 +0000,nypost,"40-year-old stabbed, officer assaulted after b...",https://nypost.com/2026/01/19/us-news/brawl-br...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
